# AI Driven Archaeological Site Mapping

This project uses Deep Learning techniques to detect archaeological
features such as ruins and vegetation from aerial and satellite images.
The goal is to assist archaeologists by reducing manual field surveys
and improving site discovery using AI.


In [ ]:
import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt


## Dataset

- Images collected from aerial and satellite sources
- Classes used:
  - Ruins
  - Vegetation
- Dataset split into:
  - Training set
  - Validation set


In [ ]:
DATASET_PATH = "/content/yolo_dataset"
TRAIN_PATH = os.path.join(DATASET_PATH, "images/train")
VAL_PATH = os.path.join(DATASET_PATH, "images/val")

print("Train path:", TRAIN_PATH)
print("Validation path:", VAL_PATH)


## Model Used

YOLOv5 is used for object detection because it provides:
- Real-time detection
- High accuracy
- Efficient performance on aerial imagery


## Model Training

The YOLOv5 model is trained using annotated archaeological images.
The model learns to identify ruins and vegetation using bounding boxes.


In [ ]:
!python train.py --img 640 --batch 16 --epochs 50 \
--data data.yaml --weights yolov5s.pt


## Model Testing / Detection

After training, the model is tested on unseen images to detect
archaeological features.


In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt \
--img 640 --source test.jpg


In [ ]:
img = cv2.imread("runs/detect/exp/test.jpg")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")


## Results

- The model successfully detected archaeological ruins.
- Detection accuracy improves with higher-resolution images.
- Vegetation and ruin areas are clearly identified using bounding boxes.


## Conclusion

AI-driven archaeological site mapping can significantly reduce
manual surveying time. Deep Learning models like YOLOv5 help
archaeologists identify potential sites efficiently and accurately
using aerial and satellite imagery.
